In [ ]:
import pandas as pd

In [ ]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [ ]:
# データの読み込み
train = pd.read_csv("../../kaggle/input/map-charting-student-math-misunderstandings/train.csv")
print(train.shape)
train.head()

## データの確認

In [ ]:
# QuestionTextの確認
print(train["QuestionText"].nunique())
print(train["QuestionText"].unique())

In [ ]:
# Categoryの確認
print(train["Category"].nunique(dropna=False))
print(train["Category"].unique())

- True_Correct
  - 意味: 生徒の説明は論理的に正しく、正しい答えと一致しています (例: 正しい推論と正しい計算)。
  - 例: 生徒が 3/9 を 1/3 に正しく簡略化し、明確に説明します。

- True_Neither
  - 意味: 生徒の説明は、明確に正しいか間違っているわけではなく、漠然としていたり​​関連性がなかったりするかもしれませんが、誤解は含まれていません。
  - 例: 生徒が理由も述べずに「それが私の理解したことです」と答えた場合、これは役に立ちませんが、間違いではありません。

- True_Misconception
  - 意味: 生徒は正しい答えを得ましたが、間違った理由で、誤解していることを示しています。
  - 例: 生徒は正解にたどり着きましたが、それは 3 つの部分が塗りつぶされていないのではなく 3 つの部分が塗りつぶされているからだと解釈しました。

- False_Neither
  - 意味: 答えは正しくなく、説明も曖昧または役に立たないが、明らかな誤解に基づいているわけではない。
  - 例: 生徒が、間違いの原因を説明しない、主題から外れた内容や数学的でない内容を書きます。

- False_Misconception
  - 意味: 回答は誤りであり、説明には特定の識別可能な誤解が示されています。
  - 例: 生徒は、9 個のうち 3 個が塗りつぶされているので (質問では塗りつぶされていないものを尋ねているのに)、9 個のうち 3 個が答えであると述べており、尋ねられていることを誤解しています。

- False_Correct
  - 意味: 生徒は間違った答えを出していますが、説明は正しいです。

In [ ]:
# Misconceptionの確認
print(train["Misconception"].nunique(dropna=False))
print(train["Misconception"].unique())

In [ ]:
# StudentExplanationの確認
print(train["StudentExplanation"].nunique())

In [ ]:
text_list = train["QuestionText"].unique()

for text in text_list:
    tmp = train[train["QuestionText"] == text]
    print(text)
    display(tmp.groupby("MC_Answer")[["Category", "Misconception"]].value_counts(dropna=False).to_frame())

## 不可解なデータの確認

### Wrong_Fraction と Wrong_fraction の意味は同じ異なるMisconceptionが存在する

In [ ]:
tmp1 = train[train["Misconception"] == "Wrong_Fraction"]
tmp2 = train[train["Misconception"] == "Wrong_fraction"]
print(tmp1.shape)
display(tmp1.head(2))
print(tmp2.shape)
display(tmp2.head(2))

In [ ]:
print(f"Wrong_Fraction: {tmp1['QuestionId'].unique()}")
print(f"Wrong_fraction: {tmp2['QuestionId'].unique()}")

問題が違うので前処理で統一して、後処理で分岐させるやり方で良いかもしれない  
というか意味が分かりづらいMisconceptionも多くあるので、対応表を作るのもありかも

### WNB と Whole_numbers_larger も同じカテゴリなのでは疑惑

In [ ]:
tmp1 = train[train["Misconception"] == "WNB"]
tmp2 = train[train["Misconception"] == "Whole_numbers_larger"]
print(tmp1.shape)
display(tmp1.head(2))
print(tmp2.shape)
display(tmp2.head(2))

In [ ]:
print(f"WNB: {tmp1['QuestionId'].unique()}")
print(f"Whole_numbers_larger: {tmp2['QuestionId'].unique()}")

In [ ]:
# シャッフル
tmp1 = tmp1.sample(frac=1).reset_index(drop=True)
tmp2 = tmp2.sample(frac=1).reset_index(drop=True)

for i in range(10):
    print(tmp1.iloc[i]["QuestionText"])
    print(tmp1.iloc[i]["MC_Answer"])
    print(tmp1.iloc[i]["Category"])
    print(tmp1.iloc[i]["StudentExplanation"])
    print(tmp1.iloc[i]["Misconception"])
    print("=" * 50)

for i in range(10):
    print(tmp2.iloc[i]["QuestionText"])
    print(tmp2.iloc[i]["MC_Answer"])
    print(tmp2.iloc[i]["Category"])
    print(tmp2.iloc[i]["StudentExplanation"])
    print(tmp2.iloc[i]["Misconception"])
    print("=" * 50)

1. WNB
これは "Wrong Number of Both"（または "Wrong numerator/denominator balance" のような略語）として使われているケースが多いです。  
与えられた分数の分母・分子を正しく数えていない、あるいは全体と一部の関係を誤解しているときに付けられているラベルです。  
例：「9個に分かれているのに、6と3を別々にして3/6と答える」→ WNB。  
つまり「全体の数え方を間違えている」誤答に対応します。

2. Whole_numbers_larger
こちらは「小数や分数よりも整数（whole number）の方が大きい」と誤解しているケースに付けられています。  
例：「どの数が大きいか？」で 6, 0.8, 0.75 があった場合に「6は整数だから一番大きい」と誤解する。  
これは「数の大小関係の誤概念」に関するラベルです。

3. 違いのまとめ
WNB → 「分数の分母・分子の扱いを誤っている（全体と部分の数え間違い）」  
Whole_numbers_larger → 「整数の方が常に小数より大きいと誤解している」  
したがって、両者は別の誤答カテゴリーであり、同じ意味ではありません。

### 特定の問題でラベル付与が間違えている

In [ ]:
id_list = [12518, 12878, 14273, 14280, 14305, 14418]
train[train["row_id"].isin(id_list)]

12787はTrueの回答、14280,14305,14418はFalseの回答のはず

In [ ]:
# 18個のラベル付与が間違えている
tmp = train.copy()
tmp = tmp[tmp["QuestionId"] == 31778]
tmp["Correct"] = tmp["Category"].str.startswith("True")
display(tmp[(tmp["MC_Answer"] == r"\( 6 \)") & (tmp["Correct"] == False)])
display(tmp[(tmp["MC_Answer"] != r"\( 6 \)") & (tmp["Correct"] == True)])

In [ ]:
# シャッフル
id_list = [
    12878,
    12901,
    13876,
    14089,
    14159,
    14185,
    14280,
    14305,
    14321,
    14335,
    14338,
    14352,
    14355,
    14403,
    14407,
    14412,
    14413,
    14418,
]
tmp1 = train[train["row_id"].isin(id_list)]
tmp2 = train[train["QuestionId"] == 31778]
tmp1 = tmp1.sample(frac=1).reset_index(drop=True)
tmp2 = tmp2.sample(frac=1).reset_index(drop=True)

print("ラベル付与誤りあり")
for i in range(16):
    print(tmp1.iloc[i]["QuestionText"])
    print(tmp1.iloc[i]["MC_Answer"])
    print(tmp1.iloc[i]["Category"])
    print(tmp1.iloc[i]["StudentExplanation"])
    print(tmp1.iloc[i]["Misconception"])
    print("=" * 50)

print("\nラベル付与誤りなし")
for i in range(100):
    print(tmp2.iloc[i]["QuestionText"])
    print(tmp2.iloc[i]["MC_Answer"])
    print(tmp2.iloc[i]["Category"])
    print(tmp2.iloc[i]["StudentExplanation"])
    print(tmp2.iloc[i]["Misconception"])
    print("=" * 50)

ラベル付与が誤ってしまった主な理由
考えられる誤りの原因は、主に以下の3つのパターンに分類できます。

1. 「解答」と「理由」の評価の混同  
これが最も多い誤りのパターンだと考えられます。生徒が書いた「理由」には正しい答えや計算過程が含まれているにもかかわらず、選択した「解答」そのものが間違っているケースです。  
具体例:  
解答: 9  
ラベル: True_Correct  
理由: I divided 9/15 by 3, then got 3/5 and timsed it by 2 and got 6/10.  
この例では、生徒は理由の中で「6/10」という正しい答えを導き出しています。しかし、最終的に選択した「解答」は「9」で、これは間違いです。  
誤りのきっかけ:  
ラベルを付与した担当者が、生徒が書いた理由の正しさに注目しすぎて、「思考プロセスは正しい (Correct)」そして「（プロセスが正しいから）正解 (True)」と判断してしまった可能性があります。本来は、まず「解答」が間違っているので「False」に分類すべきところです。  

2. 正解・不正解の単純な判断ミス  
明らかな正解を「False（不正解）」としたり、不正解を「True（正解）」としたりしているケースです。  
具体例:  
解答: 6  
ラベル: False_Neither  
理由: i worked it out on my white board  
この問題の正しい答えは 6 です。解答は合っているにもかかわらず、ラベルは「False（不正解）」となっています。  
誤りのきっかけ:  
これは、単純な見間違いやクリックミスといったヒューマンエラーである可能性が最も高いです。特に理由が書かれていない場合などに、誤って不正解と判断してしまったのかもしれません。  

3. 複雑な理由の解釈による判断の揺れ  
生徒の理由が支離滅裂であったり、部分的にしか合っていなかったりする場合、評価者の解釈が難しくなり、結果として不適切なラベルが付いてしまうことがあります。  
具体例:  
解答: 9  
ラベル: True_Neither  
理由: Because 10 is 2 / 3 of 15, and 2 is 6.  
この理由では、正しい答えである「6」という数字が出てきますが、それがなぜ解答の「9」に結びつくのか不明です。評価者が「正しい数字に言及している」という点だけを見て、不正解であるにもかかわらず「True」と付けてしまった可能性があります。

### ラベル付与誤りの要因分析表

| # | 解答 | ラベル | 理由 | 分析と割り当て要因 |
| :-- | :--- | :--- | :--- | :--- |
| 1 | `9` | True\_Correct | `i think this is because 9/15=18/30 and 6/10 =18-30.` | **要因1**: 解答は不正解だが、理由に正しい計算過程が含まれているため、理由を評価して`True`と判断してしまった。 |
| 2 | `9` | True\_Correct | `It is six because they are both equal to 3over5.` | **要因1**: 理由の中で「答えは6」と明言しており、解答の選択ミスを無視して理由の正しさを評価した典型例。 |
| 3 | `6` | False\_Neither | `m'y dad andave i bothe guessed... i wasn't ' t sur how that woulld work...` | **要因1**: 解答は正解だが、理由が「自信がない」という内容のため、理解度を低く見積もり`False`と評価した可能性が高い。 |
| 4 | `6` | False\_Neither | `Because when I work is out with rthe bar methood che awser comed up with 9.` | **要因1**: 解答は正解（6）だが、理由では「答えは9になった」と述べている。この間違った理由を評価し`False`とした。 |
| 5 | `6` | False\_Neither | `i worked it out on my white board` | **要因2**: 解答は正解だが`False`になっている。理由に判断材料がないため、単純な正誤の判断ミスと考えられる。 |
| 6 | `6` | False\_Neither | `Because on the second fraction it has a 9.` | **要因1**: 解答は正解だが、理由が的を射ていないため、思考プロセスが誤っていると判断し`False`とした可能性。 |
| 7 | `9` | True\_Correct | `To get a denominator of 10, we need to divide by 3 and multiply by 2. Then, 9/15=3/5=6/10, so A = 6.` | **要因1**: 理由が完璧に正しく「A = 6」と結論付けているため、解答の選択ミスを見逃し、理由だけを評価した例。 |
| 8 | `9` | True\_Correct | `I divided 9/15 by 3, then got 3/5 and timsed it by 2 and got 6/10.` | **要因1**: 解答は不正解だが、理由の中で正しい答え「6/10」を導いているため、理由の正しさを評価してしまった。 |
| 9 | `6` | False\_Neither | `tha answear is d becase yoy can ' t simplify 9.` | **要因1**: 解答は正解だが、理由が誤っているため、思考プロセスを重視して`False`と評価したと考えられる。 |
| 10 | `9` | True\_Neither | `Il believe that is the ansewer because I calculatted iti.` | **要因2**: 解答は不正解だが`True`になっている。理由に判断材料がないため、単純な正誤の判断ミスと考えられる。 |
| 11 | `9` | True\_Neither | `if you simplify it to 3/5 then you get 9/15.` | **要因1**: 解答は不正解だが、理由に「3/5に単純化する」という正しいプロセスが含まれるため、それを評価した可能性。 |
| 12 | `9` | True\_Correct | `I think it's C because 6/10 is the same as 9/15.` | **要因1**: 理由の中で正しい答え（C=6）と、その根拠を明確に述べているため、解答の選択ミスを無視して理由を評価した。 |
| 13 | `9` | True\_Neither | `You have to change the denominator to 150 then you will get the answer.` | **要因1**: 解答は不正解だが、理由が有効な解法アプローチに言及しているため、その点を評価してしまった可能性。 |
| 14 | `9` | True\_Neither | `since 9 - 3 = 6h are so i't must be these ohne!` | **要因3**: 理由は支離滅裂だが、偶然正しい答えの数字「6」を含む。この数字に評価者が混乱し、不適切な判断をした可能性。 |
| 15 | `9` | True\_Correct | `so the common denominator is 30 and the product of 15x2=30... therefore, a=6.` | **要因1**: 理由が非常に詳細かつ正確に「a=6」と結論付けている。解答の選択ミスより、この完璧な理由を優先して評価した例。 |
| 16 | `6` | False\_Neither | `becose thirty tope number is a nine.` | **要因1**: 解答は正解だが、理由が全く意味をなしていない。この誤った（無意味な）理由を評価し`False`とした。 |

割と回答理由だけを見て正誤評価を下していそうな問題がちらほら。おそらくテストデータにもこのようなミスが入っているはずなので、ここらへんをうまく盛り込められれば上位に食い込めるかもしれない。

### 同じ回答根拠なのに付与されるカテゴリが異なる

In [ ]:
train[train["StudentExplanation"] == "Because there are 9 triangles and 3 of them are not shaded"]

上がcorrectなのであれば、下はFalse_Correctが妥当なはず  
ここらへんの説明付ができるとよいか難しそう。LLMの理解力に任せるか